In [1]:
import cdsapi
import zipfile
import xarray as xr 
import rioxarray as rio 
import os
from tqdm import tqdm
import pandas as pd
from datetime import datetime as dt
import datetime
import rasterio
from rasterio.plot import show
from tqdm import tqdm
import numpy as np
import schedule

# Download

Data are downloaded from AgERA5 provided by CDS (https://cds.climate.copernicus.eu/cdsapp#!/dataset/sis-agrometeorological-indicators?tab=overview), through the cdsapi package. This has the advantage of magaging the caching of already passed requests, thus to speed up the downloading process.

Frequency update : daily with a 7-day lag (https://confluence.ecmwf.int/display/CUSF/AgERA-5+frequency+update)

envisager https://docs.python.org/3/library/pathlib.html

In [2]:
def download_AgERA5_data(area, selected_area, variables, mode="month", query=dt.today()):
    """
    if mode "month", download all days of the month of the datetime.date passed in query
    if mode "year", downloads all days of the year of the datetime.date passed in query    
    by default, is in mode "month", and query is the current date
    """

    print("===== download_AgERA5_data =====")

    # try:
    if mode == "month":
        query_year = query.year # int
        query_month = query.strftime('%m') # str

        print("Mode 'month' acknowledged. Will download data for month",query.month,"/",query.year)
        print("Please note that last available date on AgERA5 should be",dt.today()-datetime.timedelta(days=8))

    elif mode == "year":
        query_year = query.year

        print("Mode 'year' acknowledged. Will download data for year",query.year)
        print("Please note that last available date on AgERA5 should be",(dt.today()-datetime.timedelta(days=8)).date())

    else :
        raise Exception("The mode passed ("+mode+") is incorrect. Please use either 'month' or 'year'")


    c = cdsapi.Client()

    if not os.path.exists('../data/0_downloads/'):
        os.makedirs('../data/0_downloads/')

    for variable in variables :

        request = {
                'format': 'zip',
                'day': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                    '13', '14', '15',
                    '16', '17', '18',
                    '19', '20', '21',
                    '22', '23', '24',
                    '25', '26', '27',
                    '28', '29', '30',
                    '31',
                ],
                'year': [str(query_year)],
                'variable': variable[0],
                'statistic': variable[1],
                'area': area[selected_area],
            }

        if mode == "month":
            zip_path = '../data/0_downloads/AgERA5_'+selected_area+'_'+variable[0]+'_'+variable[1]+"_"+str(query_year)+'_'+query_month+'.zip'
            request["month"] = query_month

        if mode == "year": 
            zip_path = '../data/0_downloads/AgERA5_'+selected_area+'_'+variable[0]+'_'+variable[1]+"_"+str(query_year)+'.zip'
            request["month"] = [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ]

        # la requête doit être adaptée pour cette variable
        if variable[0] == "solar_radiation_flux" :
            del request["statistic"]

        c.retrieve(
            'sis-agrometeorological-indicators',
            request,
            zip_path)

        print("Download OK")

    # except :
    #     print("/!\ Download NOT OK")

# Extraction

In [3]:
def extract_AgERA5_data(area, selected_area, variables, mode="month", query=dt.today()):
    """
    mode can be 'month' or 'year'
    uqery must be datetime.date


    """

    print("===== extract_agERA5_month =====")

    try:
        query_year = query.year # int
        query_month = query.strftime('%m') # str

        for variable in tqdm(variables) :

            if mode == "month":
                zip_path = '../data/0_downloads/AgERA5_'+selected_area+'_'+variable[0]+'_'+variable[1]+"_"+str(query_year)+'_'+query_month+'.zip'
                
            elif mode == "year":
                zip_path = '../data/0_downloads/AgERA5_'+selected_area+'_'+variable[0]+'_'+variable[1]+"_"+str(query_year)+'.zip'
            
            extraction_path = '../data/1_extraction/AgERA5_'+selected_area+'/'+str(query_year)+"/"+variable[0]+'_'+variable[1]+'/'
            
            try:
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(extraction_path)
            except:
                pass
        
        print("Extraction OK")

    except :
        print("/!\ Extraction NOT OK")

# Conversion to daily Geotiffs

In [4]:
def convert_AgERA5_netcdf_to_geotiff(area, selected_area, variables, query=dt.today()):
    """
    converts netcdf to geotiff
    must define which variable to convert, and for which year, as defined with a datetime.date
    """

    print("===== convert_AgERA5_netcdf_to_geotiff =====")

    try:

        query_year = query.year # int
        query_month = query.strftime('%m') # str
        
        for variable in variables :
            
            extraction_path = '../data/1_extraction/AgERA5_'+selected_area+'/'+str(query_year)+"/"+variable[0]+'_'+variable[1]+'/'
            nc_files = os.listdir(extraction_path)
            conversion_path = '../data/2_conversion/AgERA5_'+selected_area+'/'+variable[0]+'_'+variable[1]+'/'

            if not os.path.exists(conversion_path):
                os.makedirs(conversion_path)

            for nc_file in tqdm(nc_files) :
                # d'après https://help.marine.copernicus.eu/en/articles/5029956-how-to-convert-netcdf-to-geotiff
                nc_file_content = xr.open_dataset(os.path.join(extraction_path, nc_file))
                xarray_variable_name = list(nc_file_content.keys())[0]
                bT = nc_file_content[xarray_variable_name]
                bT = bT.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
                bT.rio.crs
                bT.rio.write_crs("epsg:4326", inplace=True)

                filename = variable[0]+"_"+variable[1]+"_"+pd.to_datetime(nc_file_content.time.values[0]).strftime('%Y')+"_"+pd.to_datetime(nc_file_content.time.values[0]).strftime('%m')+"_"+pd.to_datetime(nc_file_content.time.values[0]).strftime('%d')+".tif"
                bT.rio.to_raster(os.path.join(conversion_path, filename))


    except :
        print("/!\ Conversion to GeoTIFFs NOT OK")

# ET0 calculation

Variables necessary for Penman-Monteith calculation : DAY, LAT, ELEV, TMIN, TMAX, IRRAD, VAP, WIND (see https://github.com/ajwdewit/pcse/blob/14a6229e4374dcde0c672d1df637cffb324a79a2/pcse/util.py)

On peut utiliser # https://api.open-elevation.com/api/v1/lookup?locations=41.161758,-8.583933 pour récupérer l'altitude

In [5]:
def calculate_AgERA5_ET0_and_save(area, selected_area, variables, query=dt.today()):

    print("===== calculate_AgERA5_ET0_and_save =====")

    try:

        query_year = query.year
        query_month = query.strftime('%m')

        # tmin
        variable = variables[0]
        conversion_path_tmin = '../data/2_conversion/AgERA5_'+selected_area+"/"+variable[0]+'_'+variable[1]+'/'
        list_files_tmin = os.listdir(conversion_path_tmin)

        # tmax
        variable = variables[1]
        conversion_path_tmax = '../data/2_conversion/AgERA5_'+selected_area+"/"+variable[0]+'_'+variable[1]+'/'
        list_files_tmax = os.listdir(conversion_path_tmax)

        # irrad
        variable = variables[2]
        conversion_path_irrad = '../data/2_conversion/AgERA5_'+selected_area+"/"+variable[0]+'_'+variable[1]+'/'
        list_files_irrad = os.listdir(conversion_path_irrad)

        # vapour pressure
        variable = variables[3]
        conversion_path_vp = '../data/2_conversion/AgERA5_'+selected_area+"/"+variable[0]+'_'+variable[1]+'/'
        list_files_vp = os.listdir(conversion_path_vp)

        # wind
        variable = variables[4]
        conversion_path_wind = '../data/2_conversion/AgERA5_'+selected_area+"/"+variable[0]+'_'+variable[1]+'/'
        list_files_wind = os.listdir(conversion_path_wind)

        # tmean
        variable = variables[5]
        conversion_path_tmean = '../data/2_conversion/AgERA5_'+selected_area+"/"+variable[0]+'_'+variable[1]+'/'
        list_files_tmean = os.listdir(conversion_path_tmean)



        # testing if all folders have the same number of files
        len(list_files_tmin) == len(list_files_tmax) == len(list_files_irrad) == len(list_files_vp) == len(list_files_wind) == len(list_files_tmean)




        for i in tqdm(range(len(list_files_irrad))):

            ## on charge les arrays

            ######################## tmin
            img_tmin = rasterio.open(os.path.join(conversion_path_tmin,list_files_tmin[i]))
            arr_tmin = img_tmin.read()
            arr_tmin = arr_tmin - 273.15

            geotiff_path = '../data/3_output/AgERA5_'+selected_area+"/2m_temperature_24_hour_minimum/"
            

            if not os.path.exists(geotiff_path):
                os.makedirs(geotiff_path)

            new_dataset = rasterio.open(
                geotiff_path+"2m_temperature_24_hour_minimum_"+'_'.join(list_files_tmin[i].split("_")[-3:]),
                'w',
                driver='GTiff',
                height=arr_tmin.shape[1],
                width=arr_tmin.shape[2],
                count=1,
                dtype=arr_tmin.dtype,
                crs=img_tmin.crs,
                transform=img_tmin.transform,
            )

            new_dataset.write(arr_tmin[0,:,:], 1)
            

            ######################## tmax
            img_tmax = rasterio.open(os.path.join(conversion_path_tmax,list_files_tmax[i]))
            arr_tmax = img_tmax.read()
            arr_tmax = arr_tmax - 273.15

            geotiff_path = '../data/3_output/AgERA5_'+selected_area+"/2m_temperature_24_hour_maximum/"

            if not os.path.exists(geotiff_path):
                os.makedirs(geotiff_path)

            new_dataset = rasterio.open(
                geotiff_path+"2m_temperature_24_hour_maximum_"+'_'.join(list_files_tmax[i].split("_")[-3:]),
                'w',
                driver='GTiff',
                height=arr_tmax.shape[1],
                width=arr_tmax.shape[2],
                count=1,
                dtype=arr_tmax.dtype,
                crs=img_tmax.crs,
                transform=img_tmax.transform,
            )

            new_dataset.write(arr_tmax[0,:,:], 1)

            ######################## irrad
            # J/m²/d
            img_irrad = rasterio.open(os.path.join(conversion_path_irrad,list_files_irrad[i]))
            arr_irrad = img_irrad.read()
            # pcse needs J/m²/d, no conversion needed
            # however SARRA needs kJ/m²/d
            arr_irrad = np.round(arr_irrad / 1000,0) # .astype(int)

            # depending on the version, it may need MJ/m²/j
            # arr_irrad = np.round(arr_irrad / 1000000,0)

            geotiff_path = '../data/3_output/AgERA5_'+selected_area+"/solar_radiation_flux_daily/"

            if not os.path.exists(geotiff_path):
                os.makedirs(geotiff_path)

            new_dataset = rasterio.open(
                geotiff_path+"solar_radiation_flux_daily_"+'_'.join(list_files_irrad[i].split("_")[-3:]),
                'w',
                driver='GTiff',
                height=arr_irrad.shape[1],
                width=arr_irrad.shape[2],
                count=1,
                dtype=arr_irrad.dtype,
                crs=img_irrad.crs,
                transform=img_irrad.transform,
            )

            new_dataset.write(arr_irrad[0,:,:], 1)


            # hPa
            img_vp = rasterio.open(os.path.join(conversion_path_vp,list_files_vp[i]))
            arr_vp = img_vp.read()
            # pcse needs hPa, no conversion needed

            # m/s
            img_wind = rasterio.open(os.path.join(conversion_path_wind,list_files_wind[i]))
            arr_wind = img_wind.read()
            # pcse needs m/s, no conversion needed

            img_tmean = rasterio.open(os.path.join(conversion_path_tmean,list_files_tmean[i]))
            arr_tmean = img_tmean.read()
            arr_tmean = arr_tmean - 273.15

            geotiff_path = '../data/3_output/AgERA5_'+selected_area+"/2m_temperature_24_hour_mean/"

            if not os.path.exists(geotiff_path):
                os.makedirs(geotiff_path)

            new_dataset = rasterio.open(
                geotiff_path+"2m_temperature_24_hour_mean_"+'_'.join(list_files_tmean[i].split("_")[-3:]),
                'w',
                driver='GTiff',
                height=arr_tmean.shape[1],
                width=arr_tmean.shape[2],
                count=1,
                dtype=arr_tmean.dtype,
                crs=img_tmean.crs,
                transform=img_tmean.transform,
            )

            new_dataset.write(arr_tmean[0,:,:], 1)

            ## on calcule le ET0

            # "When solar radiation data, relative humidity data and/or wind speed data are missing,
            # ETo can be estimated using the Hargreaves ETo equation" in FAO 56

            coeff = 0.0023
            arr_ET0 = coeff * (arr_tmean + 17.8) * 0.408 * arr_irrad/1000 * (abs(arr_tmax - arr_tmin))**0.5

            ## on sauvegarde les geotiffs

            geotiff_path = '../data/3_output/AgERA5_'+selected_area+"/ET0Hargeaves/"

            if not os.path.exists(geotiff_path):
                os.makedirs(geotiff_path)
            
            # on utilise tmean pour récupérer la date dans le nom de fichier, le crs et le transform
            new_dataset = rasterio.open(
                geotiff_path+"ET0Hargreaves_"+'_'.join(list_files_tmean[i].split("_")[-3:]),
                'w',
                driver='GTiff',
                height=arr_ET0.shape[1],
                width=arr_ET0.shape[2],
                count=1,
                dtype=arr_ET0.dtype,
                crs=img_tmean.crs,
                transform=img_tmean.transform,
            )

            new_dataset.write(arr_ET0[0,:,:], 1)

    except:
        print("/!\ Calculation of ET0 NOT OK")

# Run all

In [8]:
# parameters
area = {'burkina': [16, -6, 9, 3]}
selected_area = "burkina"
variables = [
    ("2m_temperature","24_hour_minimum"),
    ("2m_temperature","24_hour_maximum"),
    ("solar_radiation_flux", "daily"),
    ("vapour_pressure", "24_hour_mean"),
    ("10m_wind_speed", "24_hour_mean"),
    ("2m_temperature","24_hour_mean"),
]

for year in tqdm(range(1995,2019)):
    query_date = datetime.date(year,1,1)

    # run
    download_AgERA5_data(area, selected_area, variables, mode="year", query=query_date)
    extract_AgERA5_data(area, selected_area, variables, mode="year", query=query_date)
    convert_AgERA5_netcdf_to_geotiff(area, selected_area, variables, query=query_date) # converts whole year
    calculate_AgERA5_ET0_and_save(area, selected_area, variables, query=query_date) # calculates whole year
    print("===== Query date",query_date,"all done ! =====")

  0%|          | 0/24 [00:00<?, ?it/s]2023-02-21 15:47:15,535 INFO Welcome to the CDS
2023-02-21 15:47:15,537 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/sis-agrometeorological-indicators
2023-02-21 15:47:15,588 INFO Request is queued


===== download_AgERA5_data =====
Mode 'year' acknowledged. Will download data for year 1995
Please note that last available date on AgERA5 should be 2023-02-13


2023-02-21 15:48:05,180 INFO Request is completed
2023-02-21 15:48:05,181 INFO Downloading https://download-0009-clone.copernicus-climate.eu/cache-compute-0009/cache/data9/dataset-sis-agrometeorological-indicators-d8bbbf34-7c66-4ba0-b960-739758ee207a.zip to ../data/0_downloads/AgERA5_burkina_2m_temperature_24_hour_minimum_1995.zip (14.7M)
2023-02-21 15:48:06,397 INFO Download rate 12.1M/s
2023-02-21 15:48:06,465 INFO Welcome to the CDS
2023-02-21 15:48:06,466 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/sis-agrometeorological-indicators
2023-02-21 15:48:06,551 INFO Request is queued


Download OK


2023-02-21 15:48:07,588 INFO Request is running
2023-02-21 15:50:00,339 INFO Request is completed
2023-02-21 15:50:00,340 INFO Downloading https://download-0012-clone.copernicus-climate.eu/cache-compute-0012/cache/data5/dataset-sis-agrometeorological-indicators-be3dd263-7b7e-48ff-bde8-c22445cc2a69.zip to ../data/0_downloads/AgERA5_burkina_2m_temperature_24_hour_maximum_1995.zip (14.7M)
2023-02-21 15:50:02,358 INFO Download rate 7.3M/s


...

Extraction OK
===== convert_AgERA5_netcdf_to_geotiff =====


100%|██████████| 365/365 [00:18<00:00, 20.05it/s]


===== calculate_AgERA5_ET0_and_save =====


100%|██████████| 24/24 [27:20:38<00:00, 4101.60s/it]

===== Query date 2018-01-01 all done ! =====
